# Preparation

## Install libraries

In [76]:
!pip install -q -U langchain langchain_openai langgraph google-search-results

## Environment settings

In [77]:
import os
from google.colab import userdata

os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

OPENAI_MODEL = "gpt-4-turbo-preview"

# Define each components

## Define the utility functions

In [78]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", system_prompt),
      MessagesPlaceholder(variable_name="messages"),
      MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
  )
  agent = create_openai_tools_agent(llm, tools, prompt)
  return AgentExecutor(agent=agent, tools=tools)

def create_supervisor(llm: ChatOpenAI, agents: list[str]):
  system_prompt = (
    "You are the supervisor over the following agents: {agents}."
    " You are responsible for assigning tasks to each agent as requested by the user."
    " Each agent executes tasks according to their roles and responds with their results and status."
    " Please review the information and answer with the name of the agent to which the task should be assigned next."
    " Answer 'FINISH' if you are satisfied that you have fulfilled the user's request."
  )

  options = ["FINISH"] + agents
  function_def = {
    "name": "supervisor",
    "description": "Select the next agent.",
    "parameters": {
      "type": "object",
      "properties": {
        "next": {
          "anyOf": [
            {"enum": options},
          ],
        }
      },
      "required": ["next"],
    },
  }

  prompt = ChatPromptTemplate.from_messages(
    [
      ("system", system_prompt),
      MessagesPlaceholder(variable_name="messages"),
      (
        "system",
        "In light of the above conversation, please select one of the following options for which agent should be act or end next: {options}."
      ),
    ]
  ).partial(options=str(options), agents=", ".join(agents))

  return (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="supervisor")
    | JsonOutputFunctionsParser()
  )

## Define the Tools

In [79]:
from langchain.tools import tool
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.messages import HumanMessage, SystemMessage

@tool("researcher")
def researcher(query: str) -> str:
  """Research by SERP API"""
  serp_api = SerpAPIWrapper()
  return serp_api.run(query)

@tool("writer")
def writer(content: str) -> str:
  """Write a blog"""
  chat = ChatOpenAI()
  messages = [
  SystemMessage(
    content="You are a blog writer specializing in IT technology. You are responsible for writing blog posts based on the content given."
            " Articles should be in markdown format."
            " You should also make it easy for the reader to read by dividing the content into sections, using tables and figures, etc."
    ),
    HumanMessage(
      content=content
    ),
  ]
  response = chat(messages)
  return response.content

## Define the Agents

In [80]:
from langchain_core.runnables import Runnable

llm = ChatOpenAI(model=OPENAI_MODEL)

def researcher_agent() -> Runnable:
  prompt = (
    "You are a researcher who uses SERP API's search engine to find the most up-to-date and correct information."
  )
  return create_agent(llm, [researcher], prompt)

def writer_agent() -> Runnable:
  prompt = (
    "You are a blog writer specializing in IT technology."
  )
  return create_agent(llm, [writer], prompt)

## Define the Nodes

In [81]:
import operator
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage

RESEARCHER = "RESEARCHER"
WRITER = "WRITER"
SUPERVISOR = "SUPERVISOR"

agents = [RESEARCHER, WRITER]

class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]
  next: str

def researcher_node(state: AgentState) -> dict:
  result = researcher_agent().invoke(state)
  return {"messages": [HumanMessage(content=result["output"], name=RESEARCHER)]}

def writer_node(state: AgentState) -> dict:
  result = writer_agent().invoke(state)
  return {"messages": [HumanMessage(content=result["output"], name=WRITER)]}

def supervisor_node(state: AgentState) -> Runnable:
  return create_supervisor(llm, agents)

## Define the Graph

In [82]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node(RESEARCHER, researcher_node)
workflow.add_node(WRITER, writer_node)
workflow.add_node(SUPERVISOR, supervisor_node)

workflow.add_edge(RESEARCHER, SUPERVISOR)
workflow.add_edge(WRITER, SUPERVISOR)
workflow.add_conditional_edges(
  SUPERVISOR,
  lambda x: x["next"],
  {
    RESEARCHER: RESEARCHER,
    WRITER: WRITER,
    "FINISH": END
  }
)

workflow.set_entry_point(SUPERVISOR)

graph = workflow.compile()

# Run

In [87]:
prompt = (
  "Please write a blog for tech companies about the LangChain."
)

for s in graph.stream({"messages": [HumanMessage(content=prompt)]}):
  if "__end__" not in s:
    print(s)
    print("----")

{'SUPERVISOR': {'next': 'RESEARCHER'}}
----
{'RESEARCHER': {'messages': [HumanMessage(content="# Unlocking the Potential of LangChain for Tech Companies\n\nIn the rapidly evolving landscape of artificial intelligence (AI) and machine learning (ML), tech companies are constantly on the lookout for groundbreaking tools and technologies that can drive innovation, enhance efficiency, and unlock new opportunities. One such technology that has recently been making waves is LangChain, a cutting-edge framework designed to leverage the power of language models for a wide range of applications. This blog post explores what LangChain is, how it works, and why it should be on the radar of every tech company aiming to stay ahead in the game.\n\n## What is LangChain?\n\nAt its core, LangChain is a framework designed to facilitate the development and deployment of applications that harness the capabilities of large language models (LLMs). It provides an abstraction layer that simplifies the integrati